# Transform


In [1]:
from spotoptim import SpotOptim
spot = SpotOptim(fun=lambda x: x, bounds=[(1, 10)])
spot.transform_value(10., 'log10')
spot.transform_value(100., 'log')
spot.transform_value(4., 'sqrt')
spot.transform_value(2., 'exp')
spot.transform_value(3., 'square')

9.0

In [2]:
from spotoptim import SpotOptim
spot = SpotOptim(fun=lambda x: x, bounds=[(1, 10)])
spot.inverse_transform_value(1.0, 'log10')
spot.inverse_transform_value(4.605170185988092, 'log')
spot.inverse_transform_value(2.0, 'sqrt')


4.0

In [3]:
from spotoptim import SpotOptim
spot = SpotOptim(fun=lambda x: x, bounds=[('red', 'green', 'blue'), (0, 10)])
spot.process_factor_bounds()
print(spot.bounds)


[(0, 2), (0, 10)]


In [4]:
from spotoptim import SpotOptim
spot = SpotOptim(fun=lambda x: x, bounds=[('red', 'green', 'blue'), (0, 10)])
spot.detect_var_type()
['factor', 'float']

['factor', 'float']

In [5]:
from spotoptim import SpotOptim
spot = SpotOptim(fun=lambda x: x, bounds=[(0.5, 10.5)], var_type=['int'])
print(spot.bounds)
spot = SpotOptim(fun=lambda x: x, bounds=[(0, 10)], var_type=['float'])
spot.bounds


[(1, 10)]


[(0.0, 10.0)]

In [6]:
from spotoptim import SpotOptim
spot = SpotOptim(fun=lambda x: x, bounds=[(1, 10), (0.1, 100)])
spot.var_trans = ['log10', 'sqrt']
spot.transform_bounds()
print(spot.bounds)

[(0.0, 1.0), (0.31622776601683794, 10.0)]
